In [ ]:
import os
import cv2
import tifffile
import numpy as np
import statistics

from tqdm import tqdm
from glob import glob
from scipy import ndimage as ndi
from matplotlib import pyplot as plt
from skimage.io import imsave
from skimage.color import rgb2gray
from skimage import (
    measure,
    morphology,
    exposure,
    filters,
    img_as_ubyte,
    feature,
)

from commons import (crop_image_box, 
                     find_center_mask, 
                     read_image_frames_seq,
                     check_colision_border,
                     rescale_arr)

In [ ]:
def build_volume_from_directory(arr_paths, is_gray=False):
    arr_images = []

    if is_gray:
        for img_path in arr_paths:

            frame = rgb2gray(tifffile.imread(img_path))
            arr_images.append(frame)
    else:
        for img_path in arr_paths:
            
            frame = tifffile.imread(img_path)
            arr_images.append(frame)

    return np.asarray(arr_images)

In [ ]:
def plot_images(images, color='gray', names=[]):
    """
        Função para plotar array de imagens, essa função não é perfeita
        mas serve bem...
    """

    if len(names) == 0:
        names = [""] * len(images)

    if len(images) == 1:
        plt.figure(figsize=(10, 8))
        plt.imshow(images[0], color)

        return plt.show()

    fig, ax = plt.subplots(1,
                           len(images),
                           figsize=(15, 20))

    for index, arr in enumerate(images):
        ax[index].imshow(arr, cmap=color)
        ax[index].set_title(names[index])

    plt.show()
    
def read_image_frames_seq(path):
    def parser_image_name(image_name):

        *_, name = image_name.split("/")
        name, *_ = name.split(".")

        return int(name)
    
    arr = []
    
    for index in glob(path + "/*"):
        try:
            image_name = parser_image_name(index)
            
            arr.append(image_name)
            
        except:
            continue
    
    image_path_sorted = sorted(arr)

    image_unique_name = lambda x: f"{path}/{x}.tif"

    return list(map(image_unique_name, image_path_sorted))


In [ ]:
def find_bighest_cluster_area(clusters):
    """
        Essa função deve receber uma imagem segmentada (Clusters)
        Retorna a área do maior cluster
    """
    regions = measure.regionprops(clusters)

    def area(item): return item.area

    return max(map(area, regions))


def find_best_larger_cluster(image_mask):

    clusters = image_mask.copy()

    if statistics.mode(clusters.flatten()):
        clusters = np.invert(clusters)

    clusters_labels = measure.label(clusters, background=0)

    cluster_size = find_bighest_cluster_area(clusters_labels)

    return morphology.remove_small_objects(
        clusters,
        min_size=(cluster_size-1),
        connectivity=8
    )

In [ ]:
def binarize_image(arr):
    return arr > filters.threshold_triangle(arr)

In [ ]:
def watershed_reparate(image):
    
    from scipy import ndimage as ndi
    from skimage.segmentation import watershed
    from skimage.feature import peak_local_max
    
    distance = ndi.distance_transform_edt(image)
    
    coords = peak_local_max(distance, footprint=np.ones((3, 3)), labels=image)
    
    mask = np.zeros(distance.shape, dtype=bool)
    
    mask[tuple(coords.T)] = True
    
    markers, _ = ndi.label(mask)
    
    labels = watershed(-distance, markers, mask=image)
    
    return labels

def apply_thresh(image, block_size=1, thresh_type=199):
    return cv2.adaptiveThreshold(image, 
                                 1,
                                 cv2.ADAPTIVE_THRESH_MEAN_C,
                                 cv2.THRESH_BINARY, 
                                 thresh_type, 
                                 block_size).astype(bool)

def make_mask_v2(image):
    
    mask = apply_thresh(image)
    
    if statistics.mode(mask.flatten()):
        mask = np.invert(mask)
        
    mask = find_best_larger_cluster(mask)
  
    if check_colision_border(mask):
        for block in range(1, 9):

            mask = apply_thresh(image, block, thresh_type=199)
            mask = find_best_larger_cluster(mask)
            
            if not check_colision_border(mask): break

    if statistics.mode(mask.flatten()):
        mask = np.invert(mask)
    

    mask = ndi.binary_fill_holes(mask)
    mask = watershed_reparate(mask) > 20
    mask = find_best_larger_cluster(mask)
    mask = morphology.closing(mask, morphology.disk(3))
    mask = ndi.binary_fill_holes(mask)
    mask = filters.gaussian(mask, sigma=1.5)
    
    return binarize_image(mask)

def make_mask(image, background, fn_equalize=None):
    
    mask = None
    
    for thresh_type in range(199, 3, -6):
        for block in range(1, 10):

            mask = apply_thresh(rescale_arr(exposure.equalize_hist(image)),
                                block, 
                                thresh_type)
            
            mask = find_best_larger_cluster(mask)

            if not check_colision_border(mask): break
        if not check_colision_border(mask): break
            
    
    if statistics.mode(mask.flatten()):
        mask = np.invert(mask)


    mask_bin = binarize_image(
        np.subtract(exposure.equalize_hist(background),
                    exposure.equalize_hist(image))
    )

    if statistics.mode(mask_bin.flatten()):
        mask_bin = np.invert(mask_bin)

    mask = find_best_larger_cluster(mask)
    mask = find_best_larger_cluster(mask_bin + mask)
    
    ## Finalização
    
    mask = morphology.closing(mask, morphology.disk(3))

    mask = ndi.binary_fill_holes(mask)

    mask = filters.gaussian(mask, sigma=1.2)
    
    return binarize_image(mask)

# **Testes**

In [ ]:
def make_mask_v2(image):
    
    mask = apply_thresh(image)
    
    if statistics.mode(mask.flatten()):
        mask = np.invert(mask)
        
    mask = find_best_larger_cluster(mask)
  
    if check_colision_border(mask):
        for block in range(1, 9):

            mask = apply_thresh(image, block, thresh_type=199)
            mask = find_best_larger_cluster(mask)
            
            if not check_colision_border(mask): break

    if statistics.mode(mask.flatten()):
        mask = np.invert(mask)
    

    mask = ndi.binary_fill_holes(mask)
    mask = watershed_reparate(mask) > 20
    mask = find_best_larger_cluster(mask)
    mask = morphology.closing(mask, morphology.disk(3))
    mask = ndi.binary_fill_holes(mask)
    mask = filters.gaussian(mask, sigma=1.5)
    
    return binarize_image(mask)

In [ ]:
def make_mask_v3(image):
    
    mask = None
    
    for block in range(1, 50):

        mask = apply_thresh(image, block, thresh_type=199)
        mask = find_best_larger_cluster(mask)
        
        if not check_colision_border(mask):
            break

    if statistics.mode(mask.flatten()):
        mask = np.invert(mask)
    
    return mask

In [ ]:
def contour_mask_alising(mask):
    from skimage.segmentation import active_contour

    x, y = find_center_mask(mask)

    s = np.linspace(0, 2 * np.pi, 400)
    r = x + 100 * np.sin(s)
    c = y + 100 * np.cos(s)
    init = np.array([r, c]).T

    snake = active_contour(filters.gaussian(mask,
                                            3, 
                                            preserve_range=True),
                           init,
                           alpha=0.015,
                           beta=10,
                           gamma=0.001)

    masc = np.zeros(mask.shape)

    for (x, y) in snake.astype('uint8'):
        masc[x, y] = 1

    masc = feature.canny(masc)
    masc = ndi.binary_fill_holes(masc)

    return masc

In [ ]:
seq_frames = read_image_frames_seq('images_masks_cropped/images/200')
seq_frames_volume = build_volume_from_directory(seq_frames)

In [ ]:
frame = seq_frames_volume[5]
frame_eq = exposure.equalize_hist(frame)
frame_adapteq = exposure.equalize_adapthist(frame, clip_limit=0.03)

In [ ]:
plot_images([frame, frame_eq, frame_adapteq])

In [ ]:
mask1 = make_mask_v3(frame)
mask2 = make_mask_v3(rescale_arr(frame_eq))
mask3 = make_mask_v3(rescale_arr(frame_adapteq))

In [ ]:
# def crop_image_box(mask):
#     safe_pixel = 1
    
#     regions = measure.regionprops(img_as_ubyte(mask))
    
#     [props] = regions
        
#     x_min, y_min, x_max, y_max = props.bbox
    
#     return mask[x_min - safe_pixel:x_max + safe_pixel,
#                 y_min - safe_pixel:y_max + safe_pixel]


# import numpy as np
# import skimage

# img = mask_cropped

# region = label(img)

# [props] = regionprops(region)

# y0,x0 = props.centroid

# rr,cc = skimage.draw.ellipse_perimeter(int(x0), 
#                                        int(y0), 
#                                        int(props.minor_axis_length * 0.5),
#                                        int(props.major_axis_length*0.5),
#                                        orientation=props.orientation)

# plt.figure(figsize=(10, 8))
# plt.plot(rr, cc, color='lightgreen')
# plt.imshow(img)
# plt.show()

# def return_best_masks(arr):
#     total = []
    
#     for mask in arr:
#         if not check_colision_border(mask):
#             total.append(mask)
#     return sum(total).astype(bool)

In [ ]:
# Principal

# out_put_folder = 'masks_result'

# for path in glob("images_masks_cropped/images/*"):

#     seq_frames = read_image_frames_seq(path)
#     seq_frames_volume = build_volume_from_directory(seq_frames)

#     image_name = path.split('/')[2]
    
#     try:
#         os.makedirs(f'{out_put_folder}/{image_name}')
#     except: pass
    
#     for index, frame in enumerate(tqdm(seq_frames_volume)):
        
#         try:

#             frame_eq = exposure.equalize_hist(frame)

#             mask1 = make_mask_v3(frame)
#             mask2 = make_mask_v3(rescale_arr(frame_eq))  

#             mask = return_best_masks([mask1, mask2])

#             mask_gaus = ndi.gaussian_filter(mask.astype('float32'), sigma=2)
#             mask_cann = feature.canny(mask_gaus, sigma=1)
#             mask_clos = morphology.closing(mask_cann.astype(int), morphology.disk(3))
#             mask_fill = ndi.binary_fill_holes(mask_clos)
            
#             if check_colision_border(mask): continue
                
#             imsave(f"{out_put_folder}/{image_name}/{index}.tif", img_as_ubyte(mask_fill))
            
#         except Exception as e: continue

In [ ]:
# seq_frames = read_image_frames_seq('images_masks_cropped/images/002')
# seq_frames_volume = build_volume_from_directory(seq_frames)

# for frame in tqdm(seq_frames_volume):

#     try:
        
#         frame_eq = exposure.equalize_hist(frame)

#         mask1 = make_mask_v3(frame)
#         mask2 = make_mask_v3(rescale_arr(frame_eq))  
        
#         mask = return_best_masks([mask1, mask2])
        
#         mask_gaus = ndi.gaussian_filter(mask.astype('float32'), sigma=2)
#         mask_cann = feature.canny(mask_gaus, sigma=1)
#         mask_clos = morphology.closing(mask_cann.astype(int), morphology.disk(3))
#         mask_fill = ndi.binary_fill_holes(mask_clos)

#         if not check_colision_border(mask):
#             plot_images([frame, mask,mask_fill])

#     except Exception as e: continue

In [ ]:
out_put_folder = 'masks_alternative'

for path in glob("images_masks_cropped/images/*"):

    seq_frames = read_image_frames_seq(path)
    seq_frames_volume = build_volume_from_directory(seq_frames, is_gray=True)
    background = np.median(seq_frames_volume, axis=0).astype(dtype=np.uint8)

    image_name = path.split('/')[2]
    
    try:
        os.makedirs(f'{out_put_folder}/{image_name}')
    except: pass
    
    for index, frame in enumerate(tqdm(seq_frames_volume)):
        
        try:
            
            mask = make_mask(frame, background)

            if statistics.mode(mask.flatten()):
                mask = np.invert(mask)

            if check_colision_border(mask):
                mask = make_mask_v2(frame)

            if check_colision_border(mask):
                mask = make_mask(rescale_arr(exposure.equalize_hist(frame)))


            imsave(f"{out_put_folder}/{image_name}/{index}.tif", img_as_ubyte(mask))
            
        except Exception as e: continue